# Trainer
千帆Python SDK 在使用[resource API实现发起训练微调](./console-finetune.ipynb)之外，还提供了Trainer API，可以更方便地实现一体化的训练微调pipeline。同时提供了状态事件回调函数的注册，通过事件分发实现训练流程状态事件的监控。


本例将基于qianfan==0.2.1展示通过Dataset加载本地数据集，并上传到千帆平台，基于LLama-2-7b进行fine-tune，直到最终完成服务发布，并最终实现服务调用的完整过程。

## 前置准备
- 初始化千帆安全认证AK、SK

In [1]:
import os 

os.environ["QIANFAN_ACCESS_KEY"] = "your_ak"
os.environ["QIANFAN_SECRET_KEY"] = "your_sk"

## 数据集加载

千帆SDK提供了数据集实现帮助我们可以快速的加载本地的数据集到内存，并通过设定DataSource数据源以保存至本地和千帆平台。

In [3]:
from qianfan.dataset import Dataset
from qianfan.trainer import LLMFinetune

# 加载本地数据集
ds: Dataset = Dataset.load(data_file="./news_digest.jsonl")
ds.list()

[[{'prompt': '请根据下面的新闻生成摘要, 内容如下:新华社受权于18日全文播发修改后的《中华人民共和国立法法》，修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。\n生成摘要如下:',
   'response': [['修改后的立法法全文公布']]}],
 [{'prompt': '请根据下面的新闻生成摘要, 内容如下:一辆小轿车，一名女司机，竟造成9死24伤。日前，深圳市交警局对事故进行通报：从目前证据看，事故系司机超速行驶且操作不当导致。目前24名伤员已有6名治愈出院，其余正接受治疗，预计事故赔偿费或超一千万元。\n生成摘要如下:',
   'response': [['深圳机场9死24伤续：司机全责赔偿或超千万']]}],
 [{'prompt': '请根据下面的新闻生成摘要, 内容如下:1月18日，习近平总书记对政法工作作出重要指示：2014年，政法战线各项工作特别是改革工作取得新成效。新形势下，希望全国政法机关主动适应新形势，为公正司法和提高执法司法公信力提供有力制度保障。\n生成摘要如下:',
   'response': [['孟建柱：主动适应形势新变化提高政法机关服务大局的能力']]}],
 [{'prompt': '请根据下面的新闻生成摘要, 内容如下:针对央视3·15晚会曝光的电信行业乱象，工信部在公告中表示，将严查央视3·15晚会曝光通信违规违法行为。工信部称，已约谈三大运营商有关负责人，并连夜责成三大运营商和所在省通信管理局进行调查，依法依规严肃处理。\n生成摘要如下:',
   'response': [['工信部约谈三大运营商严查通信违规']]}],
 [{'prompt': '请根据下面的新闻生成摘要, 内容如下:国家食药监管总局近日发布《食品召回管理办法》，明确：食用后已经或可能导致严重健康损害甚至死亡的，属一级召回，食品生产者应在知悉食品安全风险后24小时内启动召回，且自公告发布之日起10个工作日内完成召回。\n生成摘要如下:',
   'response': [['食品一级召回限24小时内启动10工作日完成']]}],
 [{'prompt': '请根据下面的新闻生成摘要, 内容如下:人民检察院刑事诉讼涉案财物管理规定明确，不得查封

In [4]:
# 保存到千帆平台
from qianfan.dataset import QianfanDataSource
from qianfan.resources.console import consts as console_consts

bos_bucket_name = "bos_bucket_name"
bos_bucket_file_path = "/data_file_path/"


# 创建千帆数据集，并上传保存
qianfan_data_source = QianfanDataSource.create_bare_dataset(
    name="sdk_trainer_ds_04",
    template_type=console_consts.DataTemplateType.NonSortedConversation,
    storage_type=console_consts.DataStorageType.PrivateBos,
    storage_id=bos_bucket_name,
    storage_path=bos_bucket_file_path,
)

ds.save(qianfan_data_source, replace_source=True)

True

In [19]:
# 对于需要在训练过程中监控每个阶段的各个节点的用户，可以通过事件回调函数来实现
from qianfan.trainer.event import Event, EventHandler
from qianfan.trainer.consts import ActionState
from qianfan.resources.console import consts as console_consts
from qianfan.trainer.configs import TrainConfig
from qianfan.trainer.base import Pipeline
from qianfan.trainer.model import Service, DeployConfig
from qianfan.resources import QfMessages
from typing import cast


testset: Dataset = Dataset.load(data_file="./news_digest_test.jsonl")
# 定义自己的EventHandler，并实现dispatch方法
class InferAfterSFT(EventHandler):
    target_action: str
    def __init__(self, target_action: str) -> None:
        super().__init__()
        self.target_action = target_action

    def dispatch(self, event: Event) -> None:
        print("receive: <", event)
        if self.target_action == event.action_id and event.action_state == ActionState.Done:
            svc = cast(Service, event.data["service"])
            print("svc", svc)
            for row in testset.list():
                msgs = QfMessages()
                msgs.append(row[0][0]["prompt"], "user")
                svc.exec({"messages":"msgs"})
                print("row infer result", row)
            
trainer = LLMFinetune(
    train_type="ERNIE-Bot-turbo-0516",
    dataset=ds,
    train_config=TrainConfig(
        epoch=1,
        learning_rate=0.00003,
        max_seq_len=4096,
        peft_type="LoRA",
    ),
    deploy_config=DeployConfig(
        name="fin_eb_04",
        replicas=1,
        pool_type=console_consts.DeployPoolType.PrivateResource,
    ),
)
eh = InferAfterSFT(target_action=trainer.ppls[0].id)
trainer.register_event_handler(eh)
trainer.run()



receive: < {"action_id": "Pipeline_Ppbl7XMXuu", "action_state": "Preceding", "description": "action_event: action[Ppbl7XMXuu], msg:", "data": {}}
receive: < {"action_id": "Pipeline_Ppbl7XMXuu", "action_state": "Running", "description": "action_event: action[Ppbl7XMXuu], msg:pipeline running", "data": {"action": "FdG0AG6Gng"}}
receive: < {"action_id": "LoadDataSetAction_FdG0AG6Gng", "action_state": "Preceding", "description": "action_event: action[FdG0AG6Gng], msg:", "data": {}}
receive: < {"action_id": "LoadDataSetAction_FdG0AG6Gng", "action_state": "Done", "description": "action_event: action[FdG0AG6Gng], msg:", "data": {"datasets": [{"id": 38039, "type": 1}]}}
receive: < {"action_id": "Pipeline_Ppbl7XMXuu", "action_state": "Running", "description": "action_event: action[Ppbl7XMXuu], msg:pipeline running", "data": {"action": "sXRN83BjSL"}}
receive: < {"action_id": "TrainAction_sXRN83BjSL", "action_state": "Preceding", "description": "action_event: action[sXRN83BjSL], msg:", "data": {}